In [1]:
from ROOT import (TCanvas, TPad, TFile, TPaveLabel, 
                  TPaveText, gROOT, TH1F, TH1D, TLegend, 
                  gStyle, TH2F, TChain, TGraphErrors, TText, gPad, gROOT, TTree)
from array import array
import datetime
import numpy as np

np.random.seed(42)
        
from keras.models import Sequential, Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate, SimpleRNN, GRU, Masking, Lambda, Reshape, Dropout, RNN
from keras.optimizers import Adagrad, SGD, RMSprop, Adam

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, LSTM, Conv1D, SimpleRNN, Concatenate
from keras import optimizers
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score
import keras.backend as K
import pickle
#import seaborn as sns
from sklearn import preprocessing
import datetime, os
from array import array

Welcome to JupyROOT 6.14/06


Using TensorFlow backend.


In [2]:
# all methods here
def clean_arrays(data, weights=None):
    new = []
    for d in data:
        #if all(di == 0. for di in d):
        if d[-1]==0.:
            continue
        new.append(d)
    out = np.array(new)
    print("before", data.shape, 'after', out.shape)
    return out

def build_array(fname, trname, features, cuts, mll_idx = 10, should_clean_arrays=True, **kwargs):
    #print(features[mll_idx])
    f = TFile.Open(fname)
    tree = f.Get(trname)
    n_events = tree.GetEntries()
    data = np.zeros((n_events, len(features)))
    for n, (event, element) in enumerate(zip(tree, data)):
        for i, feature in enumerate(features):
            try:
                val = getattr(event, feature)
            except:
                val = None
            if feature in cuts and eval("{}{}".format(val, cuts[feature])):
                element[i] = getattr(event, feature)
            elif feature in cuts:
                element = np.zeros(len(features))
                break
            elif feature == 'train_mass':
                element[i] = getattr(event, features[mll_idx])
            elif feature == 'MZ1_MZ2':
                element[i] = getattr(event, 'MZ1')-getattr(event, 'MZ2')
            else:
                element[i] = val
    return clean_arrays(data) if should_clean_arrays else data

features = ('train_mass', ## 0
            'PtL1',       ## 1
            'PtL2',       ## 2
            'PtL3',       ## 3
            'PtL4',       ## 4
            'EtaL1',      ## 5
            'EtaL2',      ## 6
            'EtaL3',      ## 7
            'EtaL4',      ## 8
            'MZ1',        ## 9
            'MZ2',        ## 10
            'MZ1_MZ2',    ## 11
            'PtZ1',       ## 12
            'PtZ2',       ## 13
            'MZZ',        ## 14
            'PtZZ',       ## 15
            'DeltaRl12',  ## 16
            'DeltaRl34',  ## 17
            'dEtal12',    ## 18
            'dEtal34',    ## 19
            'run',        ## 20
            'event',      ## 21
            'weight')     ## 22

# these are your cuts that preselect events, empty now
cuts = dict()
#cuts = {"MZZ": "<120"}

mll_idx = 9 # 10 when signal sample < 40GeV; 9 when signal >40GeV
print("have {} features".format(len(features)))
print("mZp is", features[mll_idx])

have 23 features
mZp is MZ1


In [3]:
MASS_FEATURE_INDEX = 0
def prep_mass(xbtrain, xstrain, norm=None):
    np.random.seed(42)
    new = xbtrain.copy()
    sump = sum(xstrain[:,-1])
    for d in new:
        mass = np.random.choice(xstrain[:,MASS_FEATURE_INDEX], p=1/sump*xstrain[:,-1])
        if norm:
            mass = mass / norm
        d[MASS_FEATURE_INDEX] = mass
    return new

def get_part_feature(xtrain):
    #nf = [0,1,2,3,4,5,6,7,8,11,12,13,14,15,16,17,18,19]
    nf = [0,1,2,3,4,5,6,7,8,11,12,13,14,16,17,18,19]
    xtrain = xtrain[:,nf]
    return xtrain

# For signal trees

In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import io
import numpy as np
from keras.models import load_model

model_hM = load_model("Below180/models/noptZZ_17f_hM.h5")
#model_lM = load_model("Below180/models/Below120_18features_lM.h5")

#msamp = [42,45,48,51,54,57,60,63,66,69,72,75]
#msamp = ['qqZZ', 'ggZZ', 'ggZZ_low', 'fake']
msamp = ['fake']

base = '/Users/zhuheling/ML/keras/example_hep/myTest/'
for mass in msamp:
    _file = base+'data/tree_{}.root'.format(mass)
    print("Looking at file:", _file)
    f = TFile.Open(_file)
    list_tree=f.GetListOfKeys()
    #for it,trname in enumerate(trlist):
    for key in list_tree:
        tree=key.ReadObj()
        trname=tree.GetName()
        if "tree_" not in trname: continue
        print ("Read tree:", trname, "Entries:", tree.GetEntries())
        dat = build_array(_file, trname, features, cuts, mll_idx = 9, should_clean_arrays=False)
        dat1 = dat
        dat1[:,0] = dat1[:,9]
        dat1_par = get_part_feature(dat1)
        print(dat1_par)
        pred_hM = model_hM.predict(dat1_par).ravel()   
    
        #dat2 = dat
        #dat2[:,0] = dat2[:,10]
        #dat2_par = get_part_feature(dat2)
        #pred_lM = model_lM.predict(dat2_par).ravel()  
 
        ## plot output score
        #plt.hist(pred_hM, bins=50, histtype='step', label='signal MZ1', density=True)
        #plt.hist(pred_lM, bins=50, histtype='step', label='signal MZ2', density=True)
        #plt.legend(loc=2)
        #plt.xlabel("Output score")
        #plt.ylabel("Events")
        #plt.show()

        new_basename = os.path.basename(_file.replace('.root', '_dnn.root'))
        tree = f.Get(trname)
        
        newfile = TFile('{}/DNNtree/{}'.format(base, new_basename),'UPDATE')    
        newtree = tree.CloneTree()
        newtree.SetName(trname)
    
        DNN_highM = array( 'f', [ 0. ] )
        newbranch_0 = newtree.Branch( 'DNN_highM', DNN_highM, 'DNN_highM/F' )    
        #DNN_lowM = array( 'f', [ 0. ] )
        #newbranch_1 = newtree.Branch( 'DNN_lowM', DNN_lowM, 'DNN_lowM/F' )   
        
        for i, preds in enumerate(pred_hM):
            DNN_highM[0] = preds
            #DNN_lowM[0] = preds[1]
            newbranch_0.Fill()
            #newbranch_1.Fill()

        newtree.Write()
        newfile.Close()

    f.Close()


Looking at file: /Users/zhuheling/ML/keras/example_hep/myTest/data/tree_fake.root
Read tree: tree_NOMINAL Entries: 40855
[[   96.68231711    69.47304687    32.69924219 ... -9999.
  -9999.         -9999.        ]
 [   85.5777831     29.33117578    24.99976562 ... -9999.
  -9999.         -9999.        ]
 [   35.86128791    94.44622656    22.36219922 ... -9999.
  -9999.         -9999.        ]
 ...
 [   68.36365433    37.92746875    31.70921289 ... -9999.
  -9999.         -9999.        ]
 [   52.04980381    34.13291797    23.31287109 ... -9999.
  -9999.         -9999.        ]
 [   69.61498517    28.49956836    14.81537695 ... -9999.
  -9999.         -9999.        ]]
